# Accidents corporels de la circulation routière

Pour chaque accident corporel (soit un accident survenu sur une voie ouverte à la circulation publique, impliquant au moins un véhicule et ayant fait au moins une victime ayant nécessité des soins), des saisies d’information décrivant l’accident sont effectuées par l’unité des forces de l’ordre (police, gendarmerie, etc.) qui est intervenue sur le lieu de l’accident. Ces saisies sont rassemblées dans une fiche intitulée bulletin d’analyse des accidents corporels. L’ensemble de ces fiches constitue le fichier national des accidents corporels de la circulation dit « Fichier BAAC » administré par l’Observatoire national interministériel de la sécurité routière "ONISR".

## Dataset 

In [11]:
import pandas as pd

In [12]:
caracteristiques = pd.read_csv(
    '../../dataset/caract-2023.csv',
    sep=';',
)

lieux = pd.read_csv(
    '../../dataset/lieux-2023.csv', 
    sep=';',
    dtype={'lartpc': str}
)
vehicules = pd.read_csv(
    '../../dataset/vehicules-2023.csv',
    sep=';'
    )

usagers = pd.read_csv(
    '../../dataset/usagers-2023.csv',
    sep=';'
    )

In [13]:
print(caracteristiques.shape)
caracteristiques.head()

(54822, 15)


,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long
0,202300000001,7,5,2023,06:00,1,75,75101,2,4,2,7,RUE DE RIVOLI,"48,86638600","2,32347100"
1,202300000002,7,5,2023,05:30,5,94,94080,2,1,3,6,Avenue de Paris,"48,84547782","2,42868146"
2,202300000003,7,5,2023,20:50,1,94,94022,2,3,2,1,Avenue du Général Leclerc,"48,76240000","2,40655000"
3,202300000004,6,5,2023,23:57,5,94,94078,2,1,3,5,Rue de Paris,"48,73248432","2,44687575"
4,202300000005,7,5,2023,00:50,5,94,94068,2,2,3,3,56bis Avenue Raspail,"48,78581000","2,49217000"


In [14]:
print(vehicules.shape)
vehicules.head()

(93585, 11)


,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor,occutc
0,202300000001,155 680 557,A01,1,30,0,0,5,1,1,NaN
1,202300000002,155 680 556,A01,2,7,0,1,1,1,1,NaN
2,202300000003,155 680 554,B01,1,2,0,2,1,16,1,NaN
3,202300000003,155 680 555,A01,2,7,0,2,2,15,1,NaN
4,202300000004,155 680 551,B01,1,7,0,2,9,2,4,NaN


In [15]:
print(usagers.shape)
usagers.head()

(125789, 16)


,Num_Acc,id_usager,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp
0,202300000001,203 851 184,155 680 557,A01,1,1,4,1,1978.0,5,2,0,-1,-1,-1,-1
1,202300000002,203 851 182,155 680 556,A01,1,1,1,2,1997.0,9,1,0,-1,-1,-1,-1
2,202300000002,203 851 183,155 680 556,A01,10,3,3,1,1997.0,9,0,-1,-1,2,3,1
3,202300000003,203 851 180,155 680 554,B01,1,1,3,1,1987.0,0,2,6,0,0,0,-1
4,202300000003,203 851 181,155 680 555,A01,1,1,1,2,1984.0,0,1,0,0,0,0,-1


In [19]:
print(lieux.shape)
lieux.head()

(70860, 18)


,Num_Acc,catr,voie,v1,v2,circ,nbv,vosp,prof,pr,pr1,plan,lartpc,larrout,surf,infra,situ,vma
0,202300000001,4,RUE DE RIVOLI,0,NaN,1,2,0,1,-1,-1,1,NaN,-1,2,0,1,30
1,202300000001,4,RUE SAINT FLORENTIN,0,NaN,1,1,0,1,-1,-1,1,NaN,-1,2,0,1,30
2,202300000002,3,120,0,NaN,2,3,2,1,-1,-1,1,NaN,-1,2,0,1,50
3,202300000003,3,5,0,NaN,2,4,0,1,1,0,1,NaN,-1,2,5,1,50
4,202300000003,3,87,0,NaN,2,4,0,1,1,0,1,NaN,-1,2,5,1,50


## Traitement des données

In [16]:
caract_lieux = pd.merge(caracteristiques, lieux, on='Num_Acc', how='inner')

caract_lieux_vehicules = pd.merge(caract_lieux, vehicules, on='Num_Acc', how='inner')

resultat_final = pd.merge(caract_lieux_vehicules, usagers, on=['Num_Acc', 'id_vehicule'], how='inner')

In [17]:
resultat_final.shape

(163683, 56)

In [20]:
resultat_final.head()

,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,...,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp
0,202300000001,7,5,2023,06:00,1,75,75101,2,4,...,4,1,1978.0,5,2,0,-1,-1,-1,-1
1,202300000001,7,5,2023,06:00,1,75,75101,2,4,...,4,1,1978.0,5,2,0,-1,-1,-1,-1
2,202300000002,7,5,2023,05:30,5,94,94080,2,1,...,1,2,1997.0,9,1,0,-1,-1,-1,-1
3,202300000002,7,5,2023,05:30,5,94,94080,2,1,...,3,1,1997.0,9,0,-1,-1,2,3,1
4,202300000003,7,5,2023,20:50,1,94,94022,2,3,...,3,1,1987.0,0,2,6,0,0,0,-1


In [ ]:
class PreProcessing:
    def __init__(self):
        pass

    # Method to drop rows with missing values
    def dropna(self, df: pd.DataFrame, axis=0, how=None, thresh=None, subset=None, exclude_cols=None) -> pd.DataFrame:
        if exclude_cols:
            subset = [col for col in (subset or df.columns) if col not in exclude_cols]
        return df.dropna(axis=axis, subset=subset)

    def univariate_imputation(self, df: pd.DataFrame, strategy="mean", fill_value=None, exclude_cols=None) -> pd.DataFrame:
        qualitatives = selector(dtype_include=object)(df)
        columns = [col for col in df.columns if col not in (exclude_cols or [])]
        return df.apply(
            lambda s: s.fillna(s.mode()[0]) if s.name in qualitatives and s.name in columns
            else (s.fillna(s.mean() if strategy == "mean" else s.median() if strategy == "median" else fill_value)
                  if s.name in columns else s),
            axis=0
        )

    def multivariate_imputation(self, df: pd.DataFrame, strategy="mean", fill_value=None, exclude_cols=None):
        # Placeholder for multivariate imputation logic, respecting exclude_cols
        pass


class Processing:
    def __init__(self):
        pass

    def preprocessing(self, dataset: object, method=None, strategy=None, fill_value=None, subset=['sii'], exclude_cols=None) -> pd.DataFrame:
        preprocess = PreProcessing()
        if method == "drop" and isinstance(dataset, pd.DataFrame):
            return preprocess.dropna(df=dataset, axis=0, subset=subset, exclude_cols=exclude_cols)
        elif method == "uni_imput" and isinstance(dataset, pd.DataFrame):
            return preprocess.univariate_imputation(df=dataset, strategy=strategy, fill_value=fill_value, exclude_cols=exclude_cols)
        elif method == "mul_imput" and isinstance(dataset, pd.DataFrame):
            return preprocess.multivariate_imputation(df=dataset, strategy=strategy, fill_value=fill_value, exclude_cols=exclude_cols)

    def summary(self, df: pd.DataFrame, exclude_cols=None) -> dict:
        numeric_df = df.select_dtypes(include=[np.number])
        constant_features = [c for c in numeric_df.columns if numeric_df[c].min() == numeric_df[c].max() and c not in (exclude_cols or [])]
        missing_values = df.isna().sum()
        return {"Missing Values": missing_values, "Constant Features": constant_features}

    def remove_constant_features(self, df: pd.DataFrame, exclude_cols=None) -> pd.DataFrame:
        constant_features = self.summary(df, exclude_cols=exclude_cols)["Constant Features"]
        return df.drop(columns=constant_features)

    def remove_outliers(self, df: pd.DataFrame, threshold=1.5, exclude_cols=None) -> pd.DataFrame:
        qualitatives = selector(dtype_include=object)(df)
        quantitatives = [c for c in df.columns if c not in qualitatives and c not in (exclude_cols or [])]

        def quantile(s):
            q_025, q_075 = s.quantile(.25), s.quantile(.75)
            iq = q_075 - q_025
            return ~((s > q_075 + threshold * iq) | (s < q_025 - threshold * iq))

        mask = ft.reduce(lambda x, y: x & y, [quantile(df[col]) for col in quantitatives])
        return df.loc[mask].copy()

    def standard_scaler(self, df: pd.DataFrame, exclude_cols=None) -> pd.DataFrame:
        columns = [col for col in df.columns if col not in (exclude_cols or [])]
        df[columns] = StandardScaler().fit_transform(df[columns])
        return df

    def normalize(self, df: pd.DataFrame, norm="l2", exclude_cols=None) -> pd.DataFrame:
        columns = [col for col in df.columns if col not in (exclude_cols or [])]
        df[columns] = normalize(df[columns], norm=norm)
        return df

    def _transform_categorical(self, df: pd.DataFrame, l_order: list = None, drop="first", exclude_cols=None) -> dict:
        qualitatives = selector(dtype_include=object)(df)
        qualitatives = [col for col in qualitatives if col not in (exclude_cols or [])]
        ordinal_cols = []
        if qualitatives:
            if l_order:
                if not set(l_order).issubset(set(qualitatives)):
                    raise Exception(f"{l_order} is not include to {qualitatives}")
                else:
                    ordinal_cols = [c for c in qualitatives if c in l_order]
                    ordinal_encoder = OrdinalEncoder()
                    ordinals = ordinal_encoder.fit_transform(df[ordinal_cols])
                    ordinal_mapping = ordinal_encoder.categories_

        nominal_encoder = OneHotEncoder(sparse_output=False, drop=drop)
        nominal_cols = [c for c in qualitatives if c not in ordinal_cols]
        nominals = nominal_encoder.fit_transform(df[nominal_cols])
        nominal_mapping = nominal_encoder.get_feature_names_out()

        if len(ordinal_cols):
            column_names = list(ordinal_cols) + list(nominal_mapping)
            return {"numericals": pd.DataFrame(np.hstack([ordinals, nominals]), columns=column_names),
                    "mapping": dict(zip(ordinal_cols, ordinal_mapping))}
        else:
            column_names = list(nominal_mapping)
            return {"numericals": pd.DataFrame(data=nominals, columns=column_names)}

    def df_to_numerical(self, df: pd.DataFrame, l_order: list = None, drop="first", exclude_cols=None) -> dict:
        numericals_columns = self._transform_categorical(df, l_order, drop=drop, exclude_cols=exclude_cols)
        qualitatives = selector(dtype_include=object)(df)
        qualitatives = [col for col in qualitatives if col not in (exclude_cols or [])]
        quantitatives = [c for c in df.columns if c not in qualitatives and c not in (exclude_cols or [])]
        df_transform = numericals_columns["numericals"].reset_index(drop=True).join(df[quantitatives].reset_index(drop=True), how="inner")

        if "mapping" in numericals_columns.keys():
            return {"df_transform": df_transform, "ordinal_mapping": numericals_columns["mapping"]}
        else:
            return {"df_transform": df_transform}

    def df_to_curated(self, df: pd.DataFrame, target_col: str = 'PCIAT_PCIAT_Total', sii_col: str = 'sii', pciat_cols: list = None) -> pd.DataFrame:
        def raw_curated(row):
            if pd.isnull(row[target_col]):
                return np.nan
            possible_max_score = row[target_col] + row[pciat_cols].isnull().sum() * 5
            if row[target_col] <= 30 and possible_max_score <= 30:
                return 0
            elif 31 <= row[target_col] <= 49 and possible_max_score <= 49:
                return 1
            elif 50 <= row[target_col] <= 79 and possible_max_score <= 79:
                return 2
            elif row[target_col] >= 80 and possible_max_score >= 80:
                return 3
            return np.nan

        df[sii_col] = df.apply(raw_curated, axis=1)
        return df

In [ ]:
P = PreProcessing()

In [ ]:
to_keep = [
            'an', 'mois', 'lum', 'atm', 'int', 'col', 'catr', 'surf', 'infra', 'catv', 'motor', 'sexe', 'an_nais',  
            'catu', 'trajet', 'secu1', 'secu2', 'secu3', 'grav' 
          ]

to_delete = [
            'Num_Acc', 'id_vehicule', 'id_usager', 'num_Veh', 'lat', 'long', 'adr', 'pr', 'pr1', 'V1', 'V2',
            'senc', 'occutc', 'dep', 'com', 'agg', 'locp', 'actp', 'etatp'
            ]